# Knowledge and Data: Practical Assignment 3 
## RDF Data, RDFS knowledge and inferencing 

YOUR NAME: FRANCESCA URGESE

YOUR VUNetID: fue400 (2736482)

*(If you do not provide your name and VUNetID we will not accept your submission).* 

### Learning objectives

At the end of this exercise you should be able to:

1. Access local an external data via SPARQL both from within a python programming environment and stand-alone with a GUI, such as [YASGUI](https://yasgui.triply.cc/), and this way integrate data from different sources  
2. Model your own first knowledge base, in this case an RDF Schema knowledge graph
3. Implement inference rules 

Follow this Notebook step-by-step. 

Of course, you can do the exercises in any Programming Editor of your liking. 
But you do not have to. Feel free to simply write code in the Notebook. When 
everythink is filled in and works, safe the Notebook and submit it 
as a Jupyter Notebook, i.e. with an ipynb extension. Please use as name of the 
Notebook your studentID+Assignment3.ipynb.  

Other than in courses dedicated to programming we will not evaluate the style
of the programs. But we will test your programs on other data than we provide, 
and your program should give the correct answers to those test-data as well. 

Before you start, you need to:

- **Install the *rdflib* Python package:** *pip install rdflib* (should already be installed from the previous assignment)
- **Install the *SPARQLWrapper* Python package:** *pip install SPARQLWrapper*
- **Install the free edition of the GraphDB Triplestore:** please follow this short [GraphDB tutorial](https://github.com/ucds-vu/knowledge-data-vu/blob/master/Tutorials/Preliminaries/tutorial-GraphDB.md). 

Then, add the file example-from-slides.ttl to a newly created database, say called assignment-3. 

**Note that you should have an active internet connection to run the code in this notebook.**

In [89]:
# install library
%pip install SPARQLWrapper

Note: you may need to restart the kernel to use updated packages.


## Task 1: (3 points) Integrate Local and External Data

You can integrate SPARQL queries into your Python code by using the *RDFLib* and *SPARQLWrapper* libraries. 

The following code accesses the DBPedia knowledge graph using its SPARQL endpoint, and returns the result of the SPARQL query requesting all the labels asserted to Amsterdam (test it!)  

In [90]:
# This code only works if you are online

from rdflib import Graph, RDF, RDFS, Namespace, Literal, URIRef
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?cityName
    WHERE { 
        <http://dbpedia.org/resource/Amsterdam> rdfs:label ?cityName 
    }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    print(result["cityName"]["value"])  

Amsterdam
أمستردام
حكومة أمستردام
Amsterdam
Amsterdam
Άμστερνταμ (δήμος)
Άμστερνταμ
Amsterdam
Amsterdamo
Ámsterdam
Amsterdam
Amsterdam (commune)
Amsterdam
Amstardam
Amsterdam
アムステルダム
Amsterdam
암스테르담
Amsterdam (gemeente)
Amsterdam
Amsterdam
Amesterdão
Амстердам
Amsterdam
Амстердам
阿姆斯特丹


Your task is now the following:

A: Write a SPARQL query that extracts all the cities from your local knowledge graph (constructed by loading the file example-from-slides.ttl) 

B: Find the number of inhabitants of these cities and the longitude and latitude information (if available) from DBPedia.

C: Merge the triples from example-from-slides.ttl with the information extracted from DBpedia + Save all these triples into a new file 'extended-example.ttl' + Print all triples in Turtle Syntax.

For your convenience, we already wrote the following functions that might be useful to complete this task. 
In addition, we have loaded and printed the 'example-from-slides.ttl' dataset.

In [91]:
from rdflib import Graph, RDF, Namespace, Literal, URIRef
from SPARQLWrapper import SPARQLWrapper, JSON


# Loads the data from a certain file given as input in Turtle syntax into the Graph g  
# -------------------------
def load_graph(graph, filename):
    with open(filename, 'r') as f:
        graph.parse(f, format='turtle')
        

# Prints a certain graph given as input in Turtle syntax
# -------------------------
def serialize_graph(myGraph):
     print(myGraph.serialize(format='turtle'))
        

# Saves the Graph g in Turtle syntax to a certain file given as input
# -------------------------
def save_graph(myGraph, filename):
    with open(filename, 'w') as f:
        myGraph.serialize(filename, format='turtle')
        
    
# Changes the namespace of a certain URI given as input to a DBpedia URI 
# Example: transformToDBR("http://example.com/kad2020/Amsterdam") returns "http://dbpedia.org/resource/Amsterdam"
# -------------------------
def transformToDBR(uri):
    if isinstance(uri, Literal):
        # changes the literal to uppercase so that the object with the same name refers to an object and not the string
        return uri.upper()
    components = g.namespace_manager.compute_qname(uri)
    return "http://dbpedia.org/resource/%s"%(components[2])

# -------------------------

g = Graph()
load_graph(g, 'example-from-slides.ttl')
serialize_graph(g)


# Don't forget to run this cell before continuing the task.


@prefix ex: <http://example.com/kad/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex:Netherlands a ex:Country ;
    ex:contains ex:Ijsselmeer ;
    ex:containsCity ex:Rotterdam ;
    ex:hasCapital ex:Amsterdam ;
    ex:hasName "The Netherlands" ;
    ex:neighbours ex:Belgium .

ex:hasCapital rdfs:range ex:Capital ;
    rdfs:subPropertyOf ex:containsCity .

ex:neighbours rdfs:subPropertyOf ex:closeBy .

ex:Amsterdam a ex:Capital ;
    ex:closeBy ex:Germany .

ex:Belgium a ex:Country .

ex:EuropeanCountry rdfs:subClassOf ex:Country .

ex:Germany a ex:EuropeanCountry ;
    ex:hasCapital ex:Berlin .

ex:closeBy rdfs:domain ex:Location ;
    rdfs:range ex:Location .

ex:containsCity rdfs:domain ex:Country ;
    rdfs:range ex:City ;
    rdfs:subPropertyOf ex:contains .

ex:Capital rdfs:subClassOf ex:City .

ex:City rdfs:subClassOf ex:Location .

ex:Country rdfs:subClassOf ex:Location .




### A: Write a SPARQL query that finds all the cities in the dataset

As you cannot directly use class City, you will have to find those cities in the dataset (example-from-slides.ttl) using implicit information that can be deduced from the domain and ranges of the relations (e.g. things in a hasCapital relation are capitals and a capital is a city, etc.).

Save all the cities returned from the SPARQL query into the empty set "cities". 

In [92]:
cities = set()

query = g.query(

"""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX dbo: <http://dbpedia.org/ontology/> 
PREFIX dbr: <http://dbpedia.org/resource/> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX ex: <http://example.com/kad/> 

SELECT DISTINCT ?city
    WHERE{
    {?s ex:hasCapital ?city .}
    UNION
    {?s ex:containsCity ?city .}
    UNION
    {?s ex:has_Capital ?city .}
    }
""")

for row in query:
    cities.add(row[0])

for city in cities:
    print(city) 
    
    

http://example.com/kad/Amsterdam
http://example.com/kad/Berlin
http://example.com/kad/Rotterdam


### B: For each city, find from DBpedia its longitude & latitude, and its number of inhabitants (if available)

Don't forget to adapt the namespace of the cities in your dataset when querying DBpedia, using the above function *transformToDBR(uri)*. Also note that namespaces should never use the *https* protocol.

The empty graph h should only contain the triples extracted from DBpedia, but added to the URIs with the 'ex' namespace. 
An example of a triple in h is the following triple: 
       
       ex:Amsterdam dbo:populationTotal "872680"^^xsd:nonNegativeInteger .

In [93]:
h = Graph()

ex = Namespace("http://example.com/kad/")
dbo = Namespace("http://dbpedia.org/ontology/")
geo = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")

h.bind('ex', ex)
h.bind('dbo', dbo)
h.bind('geo', geo)

for city in cities:
    the_city = transformToDBR(URIRef(city))
    print(the_city)
    sparql2 = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql2.setQuery("""
        PREFIX ex: <http://example.com/kad/>
        PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
        PREFIX dbo: <http://dbpedia.org/ontology/>
    
        SELECT DISTINCT ?long ?lat ?populationTotal
        WHERE {
            {<""" + the_city + """> geo:long ?long.}
            {<""" + the_city + """> geo:lat ?lat .}
            { <""" + the_city + """> dbo:populationTotal ?populationTotal .}
        }
    """)
    sparql2.setReturnFormat(JSON)
    the_results = sparql2.query().convert()
    for the_result in the_results["results"]["bindings"]:
        h.add((URIRef(city), geo.long, Literal(the_result["long"]["value"]) )) 
        h.add((URIRef(city), geo.lat, Literal(the_result["lat"]["value"]) ))
        h.add((URIRef(city), geo.populationTotal, Literal(the_result["populationTotal"]["value"]) ))
    
serialize_graph(h)

http://dbpedia.org/resource/Amsterdam
http://dbpedia.org/resource/Berlin
http://dbpedia.org/resource/Rotterdam
@prefix ex: <http://example.com/kad/> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .

ex:Amsterdam geo:lat "52.3667" ;
    geo:long "4.9" ;
    geo:populationTotal "872680" .

ex:Berlin geo:lat "52.52" ;
    geo:long "13.405" ;
    geo:populationTotal "3769495" .

ex:Rotterdam geo:lat "51.9167" ;
    geo:long "4.5" ;
    geo:populationTotal "651157" .




### C: Save your results

- Merge the triples from example-from-slides.ttl with the information extracted from DBpedia.
- Save all these triples into a new file 'extended-example.ttl'. It is not necessary to submit this file.
- Print all triples in Turtle Syntax.


In [94]:
# Your code here
for s, p, o in h:
    g.add((s,p,o))
save_graph(g, "extended-example.ttl")
serialize_graph(g)

@prefix ex: <http://example.com/kad/> .
@prefix ns1: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex:Netherlands a ex:Country ;
    ex:contains ex:Ijsselmeer ;
    ex:containsCity ex:Rotterdam ;
    ex:hasCapital ex:Amsterdam ;
    ex:hasName "The Netherlands" ;
    ex:neighbours ex:Belgium .

ex:hasCapital rdfs:range ex:Capital ;
    rdfs:subPropertyOf ex:containsCity .

ex:neighbours rdfs:subPropertyOf ex:closeBy .

ex:Amsterdam a ex:Capital ;
    ex:closeBy ex:Germany ;
    ns1:lat "52.3667" ;
    ns1:long "4.9" ;
    ns1:populationTotal "872680" .

ex:Belgium a ex:Country .

ex:Berlin ns1:lat "52.52" ;
    ns1:long "13.405" ;
    ns1:populationTotal "3769495" .

ex:EuropeanCountry rdfs:subClassOf ex:Country .

ex:Germany a ex:EuropeanCountry ;
    ex:hasCapital ex:Berlin .

ex:Rotterdam ns1:lat "51.9167" ;
    ns1:long "4.5" ;
    ns1:populationTotal "651157" .

ex:closeBy rdfs:domain ex:Location ;
    rdfs:range ex:Location 

## Task 2: (3 points)  Implement Basic Inferencing Rules 

In the lecture we showed that the RDFS inference rules can be used to infer new knowledge. For example, infer class membership based on _rdfs:domain_ or infer relationships between subjects and objects based on _rdfs:subPropertyOf_. 

Create rules to inference class membership based on the RDF Schema language features 
*	For example: infer that an instance belongs to a class because of domain and range restrictions
*	For example: infer that an instance belongs to a (super)class because it also belongs to a subclass

We implemented the __rdfs2__ rule. You should implement the 5 following remaining rules:  

*     (rdfs2) If G contains the triples (aaa rdfs:domain xxx.) and (uuu aaa yyy.)  then infer the triple (uuu rdf:type xxx.)
*     (rdfs3) If G contains the triples (aaa rdfs:range xxx.) and (uuu aaa vvv.) then infer the triple (vvv rdf:type xxx .)
*     (rdfs5) If G contains the triples (uuu rdfs:subPropertyOf vvv.) and (vvv rdfs:subPropertyOf xxx.) then infer the triple
(uuu rdfs:subPropertyOf xxx.) 
*     (rdfs7) If G contains the triples (aaa rdfs:subPropertyOf bbb.) and (uuu aaa yyy.) then infer the triple (uuu bbb yyy) 
*     (rdfs9) If G contains the triples (uuu rdfs:subClassOf xxx.) and (vvv rdf:type uuu.) then infer the triple
 (vvv rdf:type xxx.)   -> this one was not mentioned in the lecture, but is a very important one. 
*     (rdfs11) If G contains the triples (uuu rdfs:subClassOf vvv.) and (vvv rdfs:subClassOf xxx.) then infer the triple
(uuu rdfs:subClassOf xxx.)


Run your rule reasoner on your knowledge graph.

In [95]:
def myRDFSreasoner(myGraph):
    inferredTriples = 0
    for sbj, prd, obj in myGraph:

        # --- rdfs2 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#domain"))):
            generator = myGraph.subject_objects(URIRef(sbj))
            for s, o in generator:
                inferredTriples += 1
                print("(rdfs 2) ", s, "rdf:type", obj)
        
        
        # --- rdfs3 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#range"))):
            generator = myGraph.subject_objects(URIRef(sbj))
            for s, o in generator:
                inferredTriples += 1
                print("(rdfs 3) ", o, "rdf:type", obj)
        
        
        # --- rdfs5 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#subPropertyOf"))):
            generator = myGraph.subject_objects(URIRef("http://www.w3.org/2000/01/rdf-schema#subPropertyOf"))
            for s, o in generator:
                if obj.eq(s):
                    inferredTriples += 1
                    print("(rdfs 5) ", sbj, "rdfs:subPropertyOf", o)
        
        
        # --- rdfs7 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#subPropertyOf"))):
            generator = myGraph.subject_objects(URIRef(sbj))
            for s, o in generator:
                inferredTriples += 1
                print("(rdfs 7) ", s, obj, o)
         
        
        # --- rdfs9 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#subClassOf"))):
            generator = myGraph.subject_objects(URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"))
            for s, o in generator:
                if sbj.eq(o):
                    inferredTriples += 1
                    print("(rdfs 9) ", s, "rdf:type", obj)
        
        
        # --- rdfs11 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#subClassOf"))):
            generator = myGraph.subject_objects(URIRef("http://www.w3.org/2000/01/rdf-schema#subClassOf"))
            for s, o in generator:
                if obj.eq(s):
                    inferredTriples += 1
                    print("(rdfs 11) ", sbj, "rdfs:subClassOf", o)
        
    print("---------------------------------")
    print("Number of inferred triples:", inferredTriples)
    print("---------------------------------")
    
myRDFSreasoner(g)


(rdfs 2)  http://example.com/kad/Amsterdam rdf:type http://example.com/kad/Location
(rdfs 7)  http://example.com/kad/Netherlands http://example.com/kad/closeBy http://example.com/kad/Belgium
(rdfs 5)  http://example.com/kad/hasCapital rdfs:subPropertyOf http://example.com/kad/contains
(rdfs 7)  http://example.com/kad/Netherlands http://example.com/kad/containsCity http://example.com/kad/Amsterdam
(rdfs 7)  http://example.com/kad/Germany http://example.com/kad/containsCity http://example.com/kad/Berlin
(rdfs 3)  http://example.com/kad/Germany rdf:type http://example.com/kad/Location
(rdfs 9)  http://example.com/kad/Netherlands rdf:type http://example.com/kad/Location
(rdfs 9)  http://example.com/kad/Belgium rdf:type http://example.com/kad/Location
(rdfs 9)  http://example.com/kad/Amsterdam rdf:type http://example.com/kad/City
(rdfs 11)  http://example.com/kad/Capital rdfs:subClassOf http://example.com/kad/Location
(rdfs 2)  http://example.com/kad/Netherlands rdf:type http://example.com/

## Task 3: (2 points) Build your very own RDFS knowledge graph. 


Define a small RDF Schema vocabulary in Turtle. You can choose your own domain (e.g. movies, geography, sports) respecting all the following rules:
*	The schema should define at least 4 classes, 4 properties, and 4 instances.
*   The properties should be used to relate the instances (i.e., object-type relations)
*	The instances should be members of at least one of the 4 defined classes
*	All resources should have an rdfs:label attribute in a suitable language.

You should use (at least) the following language features of RDF and RDFS:
* 	rdf:type (or 'a')
* 	rdfs:subClassOf
* 	rdfs:subPropertyOf
* 	rdfs:domain and rdfs:range
*	rdfs:label

Be sure to define the 'rdf:' and 'rdfs:' namespace prefixes for RDF and RDF Schema in your file (perhaps have a look at http://prefix.cc)

For creating your vocabulary, you can either use a text editor, or add the axioms directly (programatically) to your Knowledge Graph as you did last week. 

Play around with the inference rules you have created in the previous task to make sure that you added some implicit knowledge, that becomes "visible" via inferencing (this will be useful for the next task). 

Finally:
- Add the knowledge you created into the RDFlib graph datastructure *myRDFSgraph*, 
- Print *myRDFSgraph* in Turtle so that we can check your "design"
- Save *myRDFSgraph* into a new file 'myRDFSgraph.ttl' (it is not necessary to submit this file)

In [101]:

from rdflib import Graph, RDF, Namespace, Literal, URIRef
from langcodes import *
standardize_tag('en')
myRDFSgraph = Graph()

EX = Namespace('http://example.com/kad0/')
myRDFSgraph.bind('ex',EX)


def serialize_graph():
    print(myRDFSgraph.serialize(format='turtle'))
    
def save_graph(filename):
    with open(filename, 'w') as f:
        myRDFSgraph.serialize(f, format='nt')
    
def load_graph(filename):
    with open(filename, 'a') as f:
        myRDFSgraph.parse(f, format='turtle')  
        
ex = Namespace("http://example.com/kad/")
owl = Namespace("http://www.w3.org/2002/07/owl#")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")


myRDFSgraph.add((ex.StephenCurry, rdfs.label, Literal("Stephen Curry", lang="en")))
myRDFSgraph.add((ex.StephenCurry, ex.PlayIn, ex.NBA))
myRDFSgraph.add((ex.LebronJames, ex.PlayedIn, ex.DavidsonWildcats))
myRDFSgraph.add((ex.StephenCurry, ex.PlaysFor, ex.GoldenStateWarriors))
myRDFSgraph.add((ex.StephenCurry, ex.Won, ex.NBAFinalsMVP))

myRDFSgraph.add((ex.KlayThompson, rdfs.label, Literal("Klay Thompson", lang="en")))
myRDFSgraph.add((ex.KlayThompson, ex.PlayIn, ex.NBA))
myRDFSgraph.add((ex.LebronJames, ex.PlayedIn, ex.WashingtonStateCougars))
myRDFSgraph.add((ex.KlayThompson, ex.PlaysFor, ex.GoldenStateWarriors))
myRDFSgraph.add((ex.NBA, rdfs.type, ex.BasketballLeague))

myRDFSgraph.add((ex.LukaDoncic, rdfs.label, Literal("Luka Doncic",lang="en")))
myRDFSgraph.add((ex.LukaDoncic, ex.PlayIn, ex.NBA))
myRDFSgraph.add((ex.LebronJames, ex.PlayedIn, ex.RealMadrid))
myRDFSgraph.add((ex.LukaDoncic, ex.Won, ex.EuroLeagueMVP))

myRDFSgraph.add((ex.LebronJames, rdfs.label, Literal("LebronJames",lang="en")))
myRDFSgraph.add((ex.LebronJames, ex.PlayIn, ex.NBA))
myRDFSgraph.add((ex.LebronJames, ex.PlayedIn, ex.MiamHeat))
myRDFSgraph.add((ex.LebronJames, ex.PlaysFor, ex.Lakers))
myRDFSgraph.add((ex.LebronJames, ex.Won, ex.NBAFinalsMVP))

myRDFSgraph.add((ex.Player, rdfs.subClassOf, ex.Team))
myRDFSgraph.add((ex.PlayedFor, rdfs.subClassOf, ex.Team))

myRDFSgraph.add((ex.PlaysFor, rdfs.range, ex.Player))
myRDFSgraph.add((ex.PlaysFor, rdfs.domain, ex.nba))
myRDFSgraph.add((ex.PlayedIn, rdfs.range, ex.PlayedFor))
myRDFSgraph.add((ex.PlayedIn, rdfs.domain, ex.nba))
myRDFSgraph.add((ex.Won, rdfs.range, ex.Award))
myRDFSgraph.add((ex.Won, rdfs.domain, ex.nba))


        
myRDFSgraph.serialize(destination = "myRDFSgraph.ttl")
serialize_graph()

print("Now let's check what we can infer from your knowledge graph...")
print("The more rules you cover, the better!")
myRDFSreasoner(myRDFSgraph)

@prefix ns1: <http://example.com/kad/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ns1:KlayThompson rdfs:label "Klay Thompson"@en ;
    ns1:PlayIn ns1:NBA ;
    ns1:PlaysFor ns1:GoldenStateWarriors .

ns1:LebronJames rdfs:label "LebronJames"@en ;
    ns1:PlayIn ns1:NBA ;
    ns1:PlayedIn ns1:DavidsonWildcats,
        ns1:MiamHeat,
        ns1:RealMadrid,
        ns1:WashingtonStateCougars ;
    ns1:PlaysFor ns1:Lakers ;
    ns1:Won ns1:NBAFinalsMVP .

ns1:LukaDoncic rdfs:label "Luka Doncic"@en ;
    ns1:PlayIn ns1:NBA ;
    ns1:Won ns1:EuroLeagueMVP .

ns1:PlayedIn rdfs:domain ns1:nba ;
    rdfs:range ns1:PlayedFor .

ns1:PlaysFor rdfs:domain ns1:nba ;
    rdfs:range ns1:Player .

ns1:StephenCurry rdfs:label "Stephen Curry"@en ;
    ns1:PlayIn ns1:NBA ;
    ns1:PlaysFor ns1:GoldenStateWarriors ;
    ns1:Won ns1:NBAFinalsMVP .

ns1:Won rdfs:domain ns1:nba ;
    rdfs:range ns1:Award .

ns1:PlayedFor rdfs:subClassOf ns1:Team .

ns1:Player rdfs:subClassOf ns1:Team .

ns1:NBA

## Task 4 (2 points) Compare local inferences with GraphDB results

Upload *myRDFSgraph.ttl* to GraphDB (check [the GraphDB tutorial](https://github.com/ucds-vu/knowledge-data-vu/blob/master/Tutorials/Preliminaries/tutorial-GraphDB.md) before starting to work with GraphDB).

Formulate two different SPARQL queries, and write a Python code that executes these queries over your GraphDB SPARQL endpoint (check example of Task 1).

**Each SPARQL query should return a _different type_ of inferred knowledge** (at least one triple that was not explicitly asserted in the graph).

Specify below next to your query (using a comment '# ...') which type of RDFS rule is the GraphDB reasoner using to infer this answer (rdfs2, rdfs3, rdfs5, rdfs7, rdfs9, rdfs11). 

In [102]:
# Get your GraphDB repository URL (setup -> repositories -> repository url) and assign it to the variable 'myEndpoint' below. 
# It should be similar to this: 

myEndpoint = "http://LAPTOP-G9DMOUTH:7200/repositories/ASSIGNMENT_GRAPH"  # KnowledgeAndData is the name of the repository
sparql = SPARQLWrapper(myEndpoint)

In [103]:
# Query 1 - Specify which RDFS rule are you testing: 

# Check example of Task 1 on how to query remote SPARQL endpoints


sparql.setQuery("""
PREFIX ex: <http://example.com/kad/> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT ?TeamName ?PlayedFor
WHERE { 
    ?TeamName rdf:type ex:nba.
    ?TeamName ex:Won ?s.
    ?TeamName ex:PlaysFor ?PlayedFor.
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    print(result['TeamName']['value'] + " Plays For " + result['PlayedFor']['value'])  




In [99]:
# Query 2 - Specify which RDFS rule are you testing: 

# Check example of Task 1 on how to query remote SPARQL endpoints

sparql.setQuery("""

""")

